# Income Prediction

### Data preprocessing & Feature extraction

Load training data

In [2]:
import os
import pandas as pd
import numpy as np

DATA_DIR = os.path.abspath('data')
training_data_filepath = os.path.join(DATA_DIR, 'tcd-ml-1920-group-income-train.csv')
df = pd.read_csv(training_data_filepath)

c:\users\fitze\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Fix and Format data

In [4]:
# Years are in order so can replace missing years with the last valid year before
df['Year of Record'] = df['Year of Record'].fillna(method='ffill')

# Replace invalid Housing situation with other
df['Housing Situation'] = df['Housing Situation'].replace([0, '0', 'nA',np.nan], 'Unknown')

# Convert years Work Experience to float and replace '#NUM!' entries with mean
df['Work Experience in Current Job [years]'] = pd.to_numeric(df['Work Experience in Current Job [years]'], errors='coerce')
df['Work Experience in Current Job [years]'].fillna(df['Work Experience in Current Job [years]'].mean(), inplace=True)

# Replace Nan Satisfation with employer with unknown
df['Satisfation with employer'].fillna('Unknown', inplace=True)

# Remove ' EUR' from addition to salary and convert to float
df['Yearly Income in addition to Salary (e.g. Rental Income)'].replace(' EUR$', '', regex=True, inplace=True)
df['Yearly Income in addition to Salary (e.g. Rental Income)'] = pd.to_numeric(
    df['Yearly Income in addition to Salary (e.g. Rental Income)'], errors='raise')

'''                                                                       
'Gender', 'Age', 'Country', 'Size of City', 'Profession',                 
   'University Degree', 'Wears Glasses', 'Hair Color', 'Body Height [cm]' 
                                                                          
clean in same way                                                         
'''  
df

,Instance,Year of Record,Housing Situation,Crime Level in the City of Employement,Work Experience in Current Job [years],Satisfation with employer,Gender,Age,Country,Size of City,Profession,University Degree,Wears Glasses,Hair Color,Body Height [cm],Yearly Income in addition to Salary (e.g. Rental Income),Total Yearly Income [EUR]
0,1,1940.0,Unknown,33,17.0,Unhappy,other,45,Afghanistan,25179,group head,No,1,Black,182,0.00,6182.05
1,2,1940.0,Unknown,25,4.9,Unhappy,female,17,Afghanistan,2278204,heavy vehicle and mobile equipment service tec...,No,0,Blond,172,0.00,6819.69
2,3,1940.0,Unknown,34,21.0,Unhappy,female,48,Afghanistan,822134,sorter,Bachelor,0,Blond,144,0.00,8663.53
3,4,1940.0,Unknown,70,18.0,Average,female,42,Albania,59477,quality control senior engineer,No,1,Brown,152,0.00,2400.64
4,5,1940.0,Unknown,51,8.0,Happy,other,15,Albania,23494,logistician,Master,1,Black,180,0.00,2816.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048569,991705,2019.0,Large Apartment,150,14.0,Happy,NaN,32,Honduras,54925,motor vehicle operator,PhD,1,Blond,188,0.00,141621.08
1048570,991706,2019.0,Unknown,130,9.0,Somewhat Happy,NaN,20,Honduras,1126546,hiring plan analyst,Bachelor,0,Black,197,0.00,1744.25
1048571,991707,2019.0,Medium Apartment,134,11.0,Average,NaN,22,Honduras,90978,procurement contracting officer,Bachelor,0,Blond,178,61353.98,254738.53
1048572,991708,2019.0,Castle,173,21.0,Somewhat Happy,NaN,54,Honduras,1207591,kindergarten teacher,Master,0,Brown,210,0.00,11182.98


### Visualisations

In [ ]:
# TODO

### Training

In [ ]:
# TODO

### Evaluation

In [ ]:
# TODO